<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Canela Orellana
- Nombre de alumno 2: Isabel Marx


### **Link de repositorio de GitHub:** https://github.com/CanelaOrellana/MDS7202-IsabelMarx-y-CanelaOrellana

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=d35fbf146c645aa9109c35fd51a1718d197dbeb95397e7f9339ba5ec56485bc3
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow.

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados.
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/lab9/'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
df_comics = pd.read_csv(path+'df_comics.csv')
df_comics_no_label = pd.read_csv(path+'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset.

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen.

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?.

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [nltk.word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados.

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [x] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [x] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [10]:
st = StemmerTokenizer()
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

ct = ColumnTransformer(
    [("MinMaxScaler", MinMaxScaler(), atributos_de_interes),("CountVectorizer", CountVectorizer(analyzer="word",tokenizer=st,ngram_range=(1,2)),'history_text')])
pipeline_ct = Pipeline([
    ("ColumnTransformer", ct)])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`.

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [x] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [x] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [x] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [11]:
from sklearn.dummy import DummyClassifier

selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", ct),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("Multinomial", MultinomialNB()),
    ]
)

dummy_pipeline = Pipeline(
    steps=[
        ("Preprocessing", ct),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("Dummy", DummyClassifier(strategy='stratified')),
    ]
)

In [12]:
#### Código aquí ####

from sklearn.model_selection import train_test_split
y = df_comics['alignment']
X = df_comics.drop(columns=['alignment'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17)

In [13]:
selection_pipeline.fit(X_train, y_train)
dummy_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('CountVectorizer',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f51e6eced10>),
                                                  'history_text')])),
                ('Selection', SelectPercentile(percentile=90)),
                ('Dummy', DummyClassifier(strategy='stratified'))])

In [14]:
y_pred_multi = selection_pipeline.predict(X_test)
y_pred_dummy = dummy_pipeline.predict(X_test)

In [15]:
from sklearn.metrics import classification_report
print('Classification report MultinomialNB')
print(classification_report(y_test, y_pred_multi))
print('Classification report DummyClassifier')
print(classification_report(y_test, y_pred_dummy))

Classification report MultinomialNB
              precision    recall  f1-score   support

         Bad       0.67      0.06      0.11       140
        Good       0.59      0.99      0.74       245
     Neutral       1.00      0.03      0.05        40

    accuracy                           0.59       425
   macro avg       0.75      0.36      0.30       425
weighted avg       0.65      0.59      0.46       425

Classification report DummyClassifier
              precision    recall  f1-score   support

         Bad       0.36      0.39      0.38       140
        Good       0.60      0.59      0.59       245
     Neutral       0.03      0.03      0.03        40

    accuracy                           0.47       425
   macro avg       0.33      0.34      0.33       425
weighted avg       0.47      0.47      0.47       425



```
¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?
```

Se observa que el Multinomial entrenado tiene valores decentes de precision para ```Bad``` y ```Good``` y una excelente precision para ```Neutral```. Mientras que los valores de precision obtenidos por el clasificador Dummy son bastante malos, siendo el mejor de ellos para la clase ```Good```, que corresponde a ```0.55```.

En cuanto a los valores de recall, en el caso del clasificador Dummy, obtiene valores muy similares a los que obtuvo en precision. Mientras que el clasificador Multinomial tiene valores muy malos para ```Bad```y ```Neutral``` y excelente recall para la clase ```Good```.

En este caso, se consideran relevantes ambas métricas, por lo que la medida de F1 es la más importante para la decisión, ya que implica la consideración de ambas.

Si quisieramos, por ejemplo, identificar villanos, en base a los score obtenidos se puede decir que:
- El DummyClassifier identifica un 29% de los villanos que existen en el dataset. Además de todos los que identifica como villanos, sólo un 27% realmente lo son.
- El clasificador MultinomialNB sólo identifica un 6% de los villanos en el dataset. Pero, de aquellos que identifica como villanos, un 67% lo son realmente, lo cual es un buen número.


## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [16]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#Se crean pipelines para cada clasificador
svm_pipeline = Pipeline(
    steps=[
        ("Preprocessing", ct),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("clf", SVC())])

knn_pipeline = Pipeline(
    steps=[
        ("Preprocessing", ct),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("clf", KNeighborsClassifier())])

rfc_pipeline = Pipeline(
    steps=[
        ("Preprocessing", ct),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("clf", RandomForestClassifier())])

In [39]:
#### Código aquí ####
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

#Se crean las grillas para buscar los mejores hiperparámetros
param_grid_svc = [
    {
        'Preprocessing__CountVectorizer__ngram_range' : [(1,1),(1,2),(1,3)],
        'Preprocessing__CountVectorizer__max_df' : [1.0, 0.5],
        'Preprocessing__CountVectorizer__min_df' : [0.1, 0.2],
        'Selection__percentile': [20, 40, 60, 80],
        'clf__C': [10, 1, 0.2]}]

param_grid_knn = [
    {
        'Preprocessing__CountVectorizer__ngram_range' : [(1,1),(1,2),(1,3)],
        'Preprocessing__CountVectorizer__max_df' : [1.0, 0.5],
        'Preprocessing__CountVectorizer__min_df' : [0.1, 0.2],
        'Selection__percentile': [20, 40, 60, 80],
        'clf__n_neighbors': [10, 5]}]

param_grid_rfc = [
    {
        'Preprocessing__CountVectorizer__ngram_range' : [(1,1),(1,2),(1,3)],
        'Preprocessing__CountVectorizer__max_df' : [1.0, 0.5],
        'Preprocessing__CountVectorizer__min_df' : [0.1, 0.2],
        'Selection__percentile': [20, 40, 60, 80],
        'clf__max_depth' : [2, 3, 5]}]


#Se crean los objetos GridSearch
hgs_svc = HalvingGridSearchCV(svm_pipeline, param_grid_svc, n_jobs=-1, cv=3,verbose=10,random_state=17)
hgs_knn = HalvingGridSearchCV(knn_pipeline, param_grid_knn, n_jobs=-1, cv=3,verbose=10,random_state=17)
hgs_rfc = HalvingGridSearchCV(rfc_pipeline, param_grid_rfc, n_jobs=-1, cv=3,verbose=10,random_state=17)



In [40]:
#Se entrena la grilla svc
hgs_svc.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 144
n_resources: 18
Fitting 3 folds for each of 144 candidates, totalling 432 fits
----------
iter: 1
n_candidates: 48
n_resources: 54
Fitting 3 folds for each of 48 candidates, totalling 144 fits
----------
iter: 2
n_candidates: 16
n_resources: 162
Fitting 3 folds for each of 16 candidates, totalling 48 fits
----------
iter: 3
n_candidates: 6
n_resources: 486
Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('Preprocessing',
                                               ColumnTransformer(transformers=[('MinMaxScaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('CountVectorizer',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7f449d9c6980>),
                                                                                'h...text')])),
                                              ('Selection',
                                               SelectPercentile(percentile=90)),
                                              ('clf', SVC())]),
                    n_jobs=-1,
                    param_grid=[{'Preprocessing__CountVectorizer__max_df': [1.0,
                                                                            0.5],
                                 'Preprocessing__CountVectorizer__min_df': [0.1,
                                                                            0.2],
                                 'Preprocessing__CountVectorizer__ngram_range': [(1,
                                                                                  1),
                                                                                 (1,
                                                                                  2),
                                                                                 (1,
                                                                                  3)],
                                 'Selection__percentile': [20, 40, 60, 80],
                                 'clf__C': [10, 1, 0.2]}],
                    random_state=17, verbose=10)

In [41]:
#Se entrena la grilla knn
hgs_knn.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 96
n_resources: 18
Fitting 3 folds for each of 96 candidates, totalling 288 fits
----------
iter: 1
n_candidates: 32
n_resources: 54
Fitting 3 folds for each of 32 candidates, totalling 96 fits
----------
iter: 2
n_candidates: 11
n_resources: 162
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 4
n_resources: 486
Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('Preprocessing',
                                               ColumnTransformer(transformers=[('MinMaxScaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('CountVectorizer',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7f449d9c6980>),
                                                                                'h...
                                               SelectPercentile(percentile=90)),
                                              ('clf', KNeighborsClassifier())]),
                    n_jobs=-1,
                    param_grid=[{'Preprocessing__CountVectorizer__max_df': [1.0,
                                                                            0.5],
                                 'Preprocessing__CountVectorizer__min_df': [0.1,
                                                                            0.2],
                                 'Preprocessing__CountVectorizer__ngram_range': [(1,
                                                                                  1),
                                                                                 (1,
                                                                                  2),
                                                                                 (1,
                                                                                  3)],
                                 'Selection__percentile': [20, 40, 60, 80],
                                 'clf__n_neighbors': [10, 5]}],
                    random_state=17, verbose=10)

In [42]:
#Se entrena la grilla rfc
hgs_rfc.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 144
n_resources: 18
Fitting 3 folds for each of 144 candidates, totalling 432 fits
----------
iter: 1
n_candidates: 48
n_resources: 54
Fitting 3 folds for each of 48 candidates, totalling 144 fits
----------
iter: 2
n_candidates: 16
n_resources: 162
Fitting 3 folds for each of 16 candidates, totalling 48 fits
----------
iter: 3
n_candidates: 6
n_resources: 486
Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('Preprocessing',
                                               ColumnTransformer(transformers=[('MinMaxScaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('CountVectorizer',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7f449d9c6980>),
                                                                                'h...
                                               SelectPercentile(percentile=90)),
                                              ('clf',
                                               RandomForestClassifier())]),
                    n_jobs=-1,
                    param_grid=[{'Preprocessing__CountVectorizer__max_df': [1.0,
                                                                            0.5],
                                 'Preprocessing__CountVectorizer__min_df': [0.1,
                                                                            0.2],
                                 'Preprocessing__CountVectorizer__ngram_range': [(1,
                                                                                  1),
                                                                                 (1,
                                                                                  2),
                                                                                 (1,
                                                                                  3)],
                                 'Selection__percentile': [20, 40, 60, 80],
                                 'clf__max_depth': [2, 3, 5]}],
                    random_state=17, verbose=10)

In [43]:
#Se encuentran los mejores hiperparámetros de cada clasificador
print('Mejores hiperparámetros svc: \n')
display(hgs_svc.best_params_)

print('Mejores hiperparámetros knn: \n')
display(hgs_knn.best_params_)

print('Mejores hiperparámetros rfc: \n')
display(hgs_rfc.best_params_)

Mejores hiperparámetros svc: 



{'Preprocessing__CountVectorizer__max_df': 1.0,
 'Preprocessing__CountVectorizer__min_df': 0.1,
 'Preprocessing__CountVectorizer__ngram_range': (1, 3),
 'Selection__percentile': 20,
 'clf__C': 0.2}

Mejores hiperparámetros knn: 



{'Preprocessing__CountVectorizer__max_df': 1.0,
 'Preprocessing__CountVectorizer__min_df': 0.1,
 'Preprocessing__CountVectorizer__ngram_range': (1, 1),
 'Selection__percentile': 60,
 'clf__n_neighbors': 10}

Mejores hiperparámetros rfc: 



{'Preprocessing__CountVectorizer__max_df': 1.0,
 'Preprocessing__CountVectorizer__min_df': 0.1,
 'Preprocessing__CountVectorizer__ngram_range': (1, 3),
 'Selection__percentile': 80,
 'clf__max_depth': 5}

```
Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.
```

- Para el clasificador SVC se encontró que la mejor combinación es:
  - CountVectorizer ```max_df: 1.0```
  - CountVectorizer ```min_df: 0.1```
  - CountVectorizer ```ngram_range: (1, 3)```
  - SelectionPercentile ```20```
  - ```C: 0.2```

- Para el clasificador KNN se encontró que la mejor combinación es:
  - CountVectorizer ```max_df: 1.0```
  - CountVectorizer ```min_df: 0.1```
  - CountVectorizer ```ngram_range: (1, 1)```
  - SelectionPercentile ```60```
  - ```n_neighbors: 10```

- Para el clasificador RandomForestClassifier se encontró que la mejor combinación es:
  - CountVectorizer ```max_df: 1.0```
  - CountVectorizer ```min_df: 0.1```
  - CountVectorizer ```ngram_range: (1, 3)```
  - SelectionPercentile ```80```
  - ```max_depth: 5```


Lo primero a notar, es que se mantienen constantes los valores para ```max_df: 1.0``` y ```min_df: 0.1``` en los 3 clasificadores. Un valor de 1.0 para max_df significa que se consideran todas las palabras, independientemente de su frecuencia en los n-gramas en cada clasificador. Por otro lado, un valor de 0.1 para min_df indica que se descartarán las palabras que aparezcan en menos del 10% de la historia. Ambos valores indican que la mayoría de los n-gramas escogidos por cada clasificador aportan a definir si un personaje es un superhéroe o un supervillano.


El ```ngram_range``` sólo cambia para el caso del KNN, que se obtiene (1, 1) en vez de (1, 3), i.e., KNN se basa en características más simples, las palabras individuales, sin considerar combinaciones de palabras más largas. Esto puede deberse a que KNN se basa en la similitud entre las instancias y puede no requerir la captura de relaciones más complejas entre las palabras. Por otro lado, para SVC y RFC puede ser útil para capturar características más complejas y relaciones más largas en el texto. En este contexto, pueden haber palabras clave o combinaciones de palabras específicas que tienen un impacto significativo en la maldad de los superhéroes.

Por otra parte, valores de SelectPercentile de 20, 60 y 80, implican que existe gran diferencia entre qué palabras y n_gramas se consideran importantes para cada clasificador. Por ejemplo, para KNN, donde se determina que solo los unigramas aportan, se tiene un SelectPercentile de 60%, mientras que para RFC, con ngram_range (1,3), se tiene un SelectionPercentile de 80%. Luego, para RFC se tiene que cerca de las 3/4 partes de los 2-gramas y 3-gramas son consideradas importantes para RFC, cuando KNN no las considera.

Respecto a los valores obtenidos para los hiperparámetros en particular de cada clasificador:

- Hiperparámetro C de SVC: Un valor de 0.2 indica una regularización moderada, lo que permite un margen de separación más amplio entre las clases y puede ayudar a evitar el sobreajuste.

- Hiperparámetro n_neighbors de KNN: un valor de 10 significa que se considerarán las etiquetas de los 10 vecinos más cercanos para realizar la clasificación.

- Hiperparámetro max_depth de RFC: una profundidad de 5 ayuda a controlar la complejidad del modelo y prevenir el sobreajuste.

Ahora veremos, cuál de los 3 clasificadores obtiene los mejores rendimientos para hacer la mejor clasificación posible en la parte siguiente

In [28]:
#Se setean los mejores hiperparámetros para SVC

svm_pipeline.set_params(Preprocessing__CountVectorizer__max_df=1.0)
svm_pipeline.set_params(Preprocessing__CountVectorizer__min_df=0.1)
svm_pipeline.set_params(Preprocessing__CountVectorizer__ngram_range=(1, 3))
svm_pipeline.set_params(Selection__percentile=20)
svm_pipeline.set_params(clf__C=0.2)

#Se setean los mejores hiperparámetros para KNN

knn_pipeline.set_params(Preprocessing__CountVectorizer__max_df=1.0)
knn_pipeline.set_params(Preprocessing__CountVectorizer__min_df=0.1)
knn_pipeline.set_params(Preprocessing__CountVectorizer__ngram_range=(1, 1))
knn_pipeline.set_params(Selection__percentile=60)
knn_pipeline.set_params(clf__n_neighbors=10)

#Se setean los mejores hiperparámetros para RFC

rfc_pipeline.set_params(Preprocessing__CountVectorizer__max_df=1.0)
rfc_pipeline.set_params(Preprocessing__CountVectorizer__min_df=0.1)
rfc_pipeline.set_params(Preprocessing__CountVectorizer__ngram_range=(1, 3))
rfc_pipeline.set_params(Selection__percentile=80)
rfc_pipeline.set_params(clf__max_depth=5)

svm_pipeline
knn_pipeline
rfc_pipeline

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('CountVectorizer',
                                                  CountVectorizer(min_df=0.1,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f51e6eced10>),
                                                  'history_text')])),
                ('Selection', SelectPercentile(percentile=80)),
                ('clf', RandomForestClassifier(max_depth=5))])

In [30]:
#Se entrenan los 3 clasificadores
svm_pipeline.fit(X_train,y_train)
knn_pipeline.fit(X_train,y_train)
rfc_pipeline.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('CountVectorizer',
                                                  CountVectorizer(min_df=0.1,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f51e6eced10>),
                                                  'history_text')])),
                ('Selection', SelectPercentile(percentile=80)),
                ('clf', RandomForestClassifier(max_depth=5))])

In [31]:
#Se clasifica con los 3 clasificadores

y_svm = svm_pipeline.predict(X_test)
y_knn = knn_pipeline.predict(X_test)
y_rfc = rfc_pipeline.predict(X_test)

In [32]:
print('Classification report SVM')
print(classification_report(y_test, y_svm))
print('\n Classification report KNN')
print(classification_report(y_test, y_knn))
print('\n Classification report RFC')
print(classification_report(y_test, y_rfc))

Classification report SVM
              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00       140
        Good       0.58      1.00      0.73       245
     Neutral       0.00      0.00      0.00        40

    accuracy                           0.58       425
   macro avg       0.19      0.33      0.24       425
weighted avg       0.33      0.58      0.42       425


 Classification report KNN
              precision    recall  f1-score   support

         Bad       0.40      0.44      0.42       140
        Good       0.61      0.68      0.65       245
     Neutral       0.00      0.00      0.00        40

    accuracy                           0.54       425
   macro avg       0.34      0.37      0.35       425
weighted avg       0.49      0.54      0.51       425


 Classification report RFC
              precision    recall  f1-score   support

         Bad       1.00      0.04      0.07       140
        Good       0.58      1.00      0.74    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

En vista de que los resultados obtenidos son terribles, se utilizará KNN ya que es el único que predice más de un tipo en los datos (buenos y malos, aunque no predice neutrales).

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [35]:
#### Código aquí ####
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # eliminar ejemplos sin historia

y_pred = knn_pipeline.predict(df_comics_no_label)

df_comics_no_label['Prediccion'] = y_pred

In [50]:
#Predicción de Vergil
df_comics_no_label[df_comics_no_label['name']=='Vergil'].Prediccion

78    Good
Name: Prediccion, dtype: object

In [45]:
#Predicción de Gorilla Girl
df_comics_no_label[df_comics_no_label['name']=='Gorilla Girl'].Prediccion

40    Bad
Name: Prediccion, dtype: object

In [46]:
#Predicción de Batcow
df_comics_no_label[df_comics_no_label['name']=='Batcow'].Prediccion

16    Bad
83    Bad
Name: Prediccion, dtype: object

Finalmente, según nuestro clasificador se obtiene que **Batcow** y Gorilla Girl son malvadas, mientras que Vergil es buene.

De todas formas, en estos resultados se debe tener en consideración que el clasificador no predecía personajes neutrales, por lo que si alguno de estos es realmente neutral, esto no se habría detectado.

De este laboratorio queremos comentar que nos gustó el acercamiento a NLP ❤️, pero creemos que aún nos falta mucho por aprender al respecto, ya que no logramos obtener muy buenos resultados en los clasificadores implementados.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>